In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [4]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.0.0
3.1


In [2]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [3]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2) ##!! changed this from problem formulation 3 to 2

In [4]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [5]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [6]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x124ab20c0>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs'), function=<function sum_over at 0x124ab20c0>)
ScalarOutcome('RfR Investment Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x124ab20c0>)
ScalarOutcome('Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x124ab20c0>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over at 0x124

In [10]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [01:48<00:00,  1.84it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [11]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,56,109.889084,1.0,0.495727,142.377135,1.0,0.198474,49.119692,1.0,0.930366,...,9,1,3,7,2,4,4,4,0,dikesnet
1,40,297.605621,1.0,0.924324,106.668654,1.5,0.944887,326.783560,1.0,0.059685,...,9,1,3,7,2,4,4,5,0,dikesnet
2,89,290.647896,1.5,0.055623,58.544044,1.5,0.686748,131.918468,1.5,0.983720,...,9,1,3,7,2,4,4,6,0,dikesnet
3,11,254.622549,1.5,0.208255,219.048653,10.0,0.333599,337.020536,1.0,0.379379,...,9,1,3,7,2,4,4,7,0,dikesnet
4,75,244.082670,10.0,0.542971,97.959127,10.0,0.702444,116.113765,1.5,0.707513,...,9,1,3,7,2,4,4,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,13,43.609953,1.5,0.864704,160.003987,1.5,0.127048,32.803669,1.0,0.205819,...,0,6,7,5,1,6,3,49,3,dikesnet
196,50,56.648318,1.5,0.139488,319.285674,10.0,0.050492,196.065119,10.0,0.082973,...,0,6,7,5,1,6,3,50,3,dikesnet
197,27,333.832041,1.0,0.600740,247.424771,1.0,0.561847,239.213635,1.5,0.765258,...,0,6,7,5,1,6,3,51,3,dikesnet
198,78,86.317839,1.0,0.707363,42.879498,10.0,0.216369,281.971545,10.0,0.917609,...,0,6,7,5,1,6,3,52,3,dikesnet


In [12]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.173742e+08,0.0,1.319753e+08,0.000434,3.139880e+07,0.000000,3.852595e+07,0.000000,1.242471e+08,0.000000,1.207400e+09,236.391152
1,2.173742e+08,0.0,1.289043e+08,0.000000,1.918928e+08,0.023344,3.852595e+07,0.000000,1.242471e+08,0.000000,1.207400e+09,7225.040591
2,2.173742e+08,0.0,1.289043e+08,0.000000,3.139880e+07,0.000000,3.852595e+07,0.000000,1.242471e+08,0.000000,1.207400e+09,0.000000
3,2.173742e+08,0.0,1.289043e+08,0.000000,3.703855e+07,0.001454,3.852595e+07,0.000000,1.242471e+08,0.000000,1.207400e+09,364.219849
4,2.173742e+08,0.0,1.289043e+08,0.000000,3.139880e+07,0.000000,3.852595e+07,0.000000,1.314744e+08,0.000978,1.207400e+09,643.740703
...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.816351e+08,0.0,1.903087e+08,0.000449,1.693374e+08,0.000000,3.003357e+07,0.000000,1.155633e+08,0.000000,1.141600e+09,205.039639
196,1.816351e+08,0.0,1.988796e+08,0.002258,1.693374e+08,0.000000,3.003357e+07,0.000000,1.155633e+08,0.000000,1.141600e+09,1052.512117
197,1.816351e+08,0.0,1.852250e+08,0.000000,1.693374e+08,0.000000,3.112300e+07,0.000069,1.155633e+08,0.000000,1.141600e+09,70.656630
198,1.816351e+08,0.0,1.891938e+08,0.000459,1.693374e+08,0.000000,3.104397e+07,0.000061,1.155633e+08,0.000000,1.141600e+09,275.167610


In [7]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"A.1_DikeIncrease 0": 0}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1, "5_RfR 0": 1, "6_RfR 0": 1}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{ "A.3_DikeIncrease 0": 10, "A.2_DikeIncrease 0": 10, "A.3_DikeIncrease 0": 10, "A.4_DikeIncrease 0": 10, "A.5_DikeIncrease 0": 10}
        )
    ),
]

In [8]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [02:56<00:00,  1.70it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
experiments, outcomes = results

In [12]:
# only works because we have scalar outcomes
df_out = pd.DataFrame(outcomes)

In [38]:
#df_out.iloc[0:100, 0].max()
#df_out.iloc[0:100, 4].max()
#(df_out.iloc[100:200, 0]+df_out.iloc[100:200, 2]).max()
#df_out.iloc[100:200, 2].max()
#df_out.iloc[100:200, 4].max()
#(df_out.iloc[200:300, 0]+df_out.iloc[200:300, 1]).max()
#df_out.iloc[200:300, 1].max()
df_out.iloc[200:300, 4].max()

1.9786142501291217

In [43]:
#saving experiment and outcome data for later analysis
experiments.to_csv('data/policy-experiments.csv')
df_out.to_csv('data/policy-outcomes.csv')

- A 'do nothing' policy would in the worst case (of 100 sampled uncertainty scenarios) cause 6 billion € in economic damage and 5 deaths per year (i.e. 135 deaths by 2050).
- A 'only RfR' policy would in the worst case (of 100 sampled uncertainty scenarios) cost 3.5 billion € total costs (damage + investment), of which 0.7 billion are investment costs, and 3 deaths per year (i.e. 81 deaths by 2050).
- A 'only Dike Raising' policy would in the worst case (of 100 sampled uncertainty scenarios) cost 3.9 billion € total costs (damage + investment), of which 0.2 billion are investment costs, and 2 deaths per year (i.e. more than 54 deaths by 2050).